In [5]:
import json
import pandas as pd
import re

In [6]:
with open('result.json', encoding='utf8') as f:
    data = json.load(f)

In [19]:
def get_msg_df(df):
    msg_df = df[df['type']=='message'].filter(items = ['id','from','text','from_id']).reset_index(drop=True)
    tmp_msg_df = []
    for index, row in msg_df.iterrows():
        if type(row['text']) == list:
            s = ''
            for item in row['text']:
                if type(item) == str:
                    s += item
                elif item['type'] != 'link':
                    s += item['text']
            row['text'] = s
        s = ''
        for c in row['text']:
            c = c.lower()
            if c.isalpha() or c.isdigit() or c == "'":
                s += c
            elif c == '?':
                s += ' '+c+' '
            else:
                s += ' '
        row['text'] = ' '.join(s.split())
        tmp_msg_df.append(row)

    tmp_msg_df = pd.DataFrame(tmp_msg_df).reset_index(drop=True)
    msg_df['clean_text'] = tmp_msg_df['text']
    msg_df = msg_df[msg_df['clean_text'] != ''].reset_index(drop=True)
    return msg_df

In [20]:
def get2_msg_df(df):

    def preprocessing_msg_df(msg_df):
        new_msg_df = []
        for index, row in msg_df.iterrows():
            if type(row['text']) == list:
                s = ''
                for item in row['text']:
                    if type(item) == str:
                        s += item
                    else:
                        s += item['text']
                row['text'] = s
            if row['text'] != '':
                new_msg_df.append(row)
        new_msg_df = pd.DataFrame(new_msg_df).reset_index(drop=True)
        return new_msg_df

    def cleaning_msg_df(msg_df):
        msg_df['clean_text'] = msg_df['text'].str.lower()
        msg_df['clean_text'] = msg_df['clean_text'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http+?", "", elem))  
        return msg_df
    
    msg_df = df[df['type']=='message'].filter(items = ['id','from','text','from_id']).reset_index(drop=True)
    msg_df = preprocessing_msg_df(msg_df)
    msg_df = cleaning_msg_df(msg_df)
    msg_df = msg_df[msg_df['clean_text'] != ''].reset_index(drop=True)
    return msg_df

In [21]:
df = pd.DataFrame(data['messages'])
msg_df = get_msg_df(df)

In [22]:
msg_df

,id,from,text,from_id,clean_text
0,28,Mr Pig,"This is 10 players Battle Royale, imagine when...",user194359594,this is 10 players battle royale imagine when ...
1,29,Legendary Harry Pottre,Here's the real question...\n\nWen beta keys,user1630446861,here's the real question wen beta keys
2,47,Apple J,GM,user1776264703,gm
3,51,K S,"Hey guys, this is Kurtney from Glock Ventures....",user656383476,hey guys this is kurtney from glock ventures w...
4,53,Aaron Brentwood,"Hey Guys,",user1866130249,hey guys
...,...,...,...,...,...
49222,117899,Nalma ~Will Never PM For Fund$~,"Anyone you wish to use, no difference between ...",user1222051294,anyone you wish to use no difference between t...
49223,117900,Oryan94,thks very much for information,user1977369058,thks very much for information
49224,117902,Jany,"""User can not log in"" this is the display mess...",user616497746,user can not log in this is the display messag...
49225,117906,Gabriele,Guys I just had a bug occurring on the android...,user59544811,guys i just had a bug occurring on the android...


In [23]:
acc_df = msg_df.groupby('from_id', as_index=False).aggregate(list)
row_df = msg_df['clean_text']

0        this is 10 players battle royale imagine when ...
1                   here's the real question wen beta keys
2                                                       gm
3        hey guys this is kurtney from glock ventures w...
4                                                 hey guys
                               ...                        
49222    anyone you wish to use no difference between t...
49223                       thks very much for information
49224    user can not log in this is the display messag...
49225    guys i just had a bug occurring on the android...
49226                                  exit and enter back
Name: clean_text, Length: 49227, dtype: object